# Boosting

In [1]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)

Windows-10-10.0.19045-SP0
Python 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


## Setup

In [2]:
import numpy as np
import pandas as pd

import os 
import joblib
import time

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

import optuna
import catboost

In [3]:
class cfg:
    modelname = 'catboost'
    debug = False
    optim = False
    seed = 42
    nfolds = 5
    njobs = 2

In [4]:
# load data
train = pd.read_csv('../data/final/train.csv')
valid = pd.read_csv('../data/final/valid.csv')
test = pd.read_csv('../data/final/test.csv')

train.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,IsSynthetic,...,Monterey County,Napa County,Orange County,Other,Riverside County,San Francisco County,Santa Barbara County,Santa Clara County,Ventura County,Yolo County
0,5.0855,17.0,6.190805,1.105747,1538.0,3.373563,38.02,-121.36,1.035,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.3636,5.0,4.239899,1.020202,3278.0,2.066007,33.68,-117.53,1.969,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0417,29.0,6.602317,1.023529,1119.0,2.705479,33.84,-117.95,2.638,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.1364,22.0,5.448584,0.986717,1750.0,3.020478,38.47,-121.77,1.188,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.6031,39.0,4.742204,1.075092,1168.0,3.250000,36.75,-119.80,0.669,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# quick info
TARGET = 'MedHouseVal'
FEATURES = [c for c in train.columns if c not in [TARGET]]

print(f'Target: {TARGET}\nFeatures: {FEATURES}')
print('Train set shape:', train.shape)
print('Validation set shape:', valid.shape)
print('Test set shape:', test.shape)

x = train[FEATURES]
y = train[TARGET]

Target: MedHouseVal
Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'IsSynthetic', 'AnomalyScore', 'rot_15_x', 'rot_15_y', 'rot_30_x', 'rot_30_y', 'rot_45_x', 'pca_lon', 'pca_lat', 'Alameda County', 'Contra Costa County', 'Fresno County', 'Kern County', 'Los Angeles County', 'Monterey County', 'Napa County', 'Orange County', 'Other', 'Riverside County', 'San Francisco County', 'Santa Barbara County', 'Santa Clara County', 'Ventura County', 'Yolo County']
Train set shape: (42922, 33)
Validation set shape: (14855, 33)
Test set shape: (24759, 32)


In [6]:
# setup
params_path = './training_files/params/'
os.makedirs(params_path, exist_ok=True)

cv = KFold(n_splits=cfg.nfolds, shuffle=True, random_state=cfg.seed)

## Hyperparameter tuning

In [7]:
# fixed params
fixed_params = {
    'loss_function': 'RMSE',
    'custom_metric': 'RMSE',
    'task_type' : 'CPU',
    'bootstrap_type': 'Bayesian',
    'allow_writing_files': False,
}

# objective function for optimization
def objective(trial):
    
    # trial parameters
    tuning_params = {
        'num_trees': trial.suggest_int('num_trees', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 1, 25),
        'random_strength': trial.suggest_float('random_strength', 1, 10),
        'depth': trial.suggest_int('depth', 1, 8, step=1),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.1, 0.8),
        'l2_leaf_reg':trial.suggest_float('l2_leaf_reg', 0, 10)
    }

    params = {**fixed_params, **tuning_params}

    # train and score with cv
    scores = []
    for train_idx, test_idx in cv.split(x, y):
        
        # split data
        x_train, x_val = x.iloc[train_idx], x.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        
        # fit model
        model = catboost.CatBoostRegressor(**params)
        model.fit(
            catboost.Pool(x_train, y_train),
            eval_set=catboost.Pool(x_val, y_val),
            early_stopping_rounds=20,
            verbose=0
        )
        
        # obtain score
        rmse = np.sqrt(mean_squared_error(y_val, model.predict(x_val)))
        scores.append(rmse)

    # return mean cv score 
    return np.mean(scores)

In [8]:
if cfg.optim:

    # create study
    sampler = optuna.samplers.TPESampler(seed=cfg.seed)
    max_trials = 2 if cfg.debug else 50
    time_limit = 3600 * 0.5

    study = optuna.create_study(
        sampler=sampler,
        study_name= f'{cfg.modelname}_optimization',
        direction='minimize')

    # perform optimization
    print(f'Starting {cfg.modelname} optimization...')
    study.optimize(
        objective,
        n_trials = max_trials,
        timeout = time_limit,
    )

In [9]:
if cfg.optim:
    
    # optimization results
    print(f"Number of finished trials: {len(study.trials)}")
    print(f"Best score: {study.best_value}")
    best_params = {**fixed_params, **study.best_trial.params}
    print("Best trial parameters:")
    for k, v in best_params.items():
        print(f"\t{k}: {v}")

    # save best params
    best_params_path = f'{params_path}{cfg.modelname}.joblib'
    with open(best_params_path, "wb") as file:
        joblib.dump(best_params, file)

## Final model

In [10]:
# load best params
best_params_path = f'{params_path}{cfg.modelname}.joblib'
with open(best_params_path, 'rb') as file:
    best_params = joblib.load(file)

best_params['learning_rate'] = 0.02
print("Final parameters:")
for k, v in best_params.items():
    print(f"\t{k}: {v}")

Final parameters:
	loss_function: RMSE
	custom_metric: RMSE
	task_type: CPU
	bootstrap_type: Bayesian
	allow_writing_files: False
	num_trees: 955
	learning_rate: 0.02
	bagging_temperature: 4.605862369391987
	random_strength: 2.1610845898091626
	depth: 8
	colsample_bylevel: 0.17170220706427092
	l2_leaf_reg: 6.345347577047418


In [11]:
# cv training and predict
def train_model(train, valid, test, params):

    start_time = time.time()
    print(30*'*', f'Model: {cfg.modelname}', 30*'*', '\n')

    # get data
    x = train[FEATURES]
    y = train[TARGET]

    # cv loop
    cv = KFold(n_splits=cfg.nfolds, shuffle=True, random_state=cfg.seed)
    preds, preds_val, cv_scores = [], [], []
    for fold, (train_idx, test_idx) in enumerate(cv.split(x, y)):

        print(f'fold {fold+1}/{cfg.nfolds}...')
        fold_start_time = time.time()

        # split data
        x_train, x_val = x.iloc[train_idx], x.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        # define and fit model
        model = catboost.CatBoostRegressor(**params)
        model.fit(
            catboost.Pool(x_train, y_train),
            eval_set=catboost.Pool(x_val, y_val),
            early_stopping_rounds=20,
            verbose=200
        )

        # validation score
        cv_score = np.sqrt(mean_squared_error(y_val, model.predict(x_val)))
        cv_scores.append(cv_score)

        # predict validation and test data
        preds_val.append(model.predict(valid[FEATURES]))
        preds.append(model.predict(test))

        fold_run_time = time.time() - fold_start_time
        print(f'rmse: {cv_score:.4f}, run time: {fold_run_time:.2f}\n')

    # print results
    run_time = time.time() - start_time
    print(f'\nTraining completed. Total run time: {run_time:2f}')
    print(f'CV score:\n\t mean: {np.mean(cv_scores):0.6f}\n\t std: {np.std(cv_scores):0.6f}')

    return (cv_scores, preds, preds_val)

In [12]:
# train model
scores, preds, preds_val = train_model(train, valid, test, best_params)

****************************** Model: catboost ****************************** 

fold 1/5...
0:	learn: 1.1450293	test: 1.1427157	best: 1.1427157 (0)	total: 150ms	remaining: 2m 23s
200:	learn: 0.5712596	test: 0.5829330	best: 0.5829330 (200)	total: 3.84s	remaining: 14.4s
400:	learn: 0.5398070	test: 0.5575769	best: 0.5575769 (400)	total: 7.42s	remaining: 10.2s
600:	learn: 0.5246856	test: 0.5472671	best: 0.5472671 (600)	total: 10.3s	remaining: 6.04s
800:	learn: 0.5136651	test: 0.5407016	best: 0.5407016 (800)	total: 13s	remaining: 2.49s
954:	learn: 0.5065356	test: 0.5368636	best: 0.5368636 (954)	total: 15.3s	remaining: 0us

bestTest = 0.5368635776
bestIteration = 954

rmse: 0.5369, run time: 15.71

fold 2/5...
0:	learn: 1.1451094	test: 1.1417398	best: 1.1417398 (0)	total: 27.6ms	remaining: 26.3s
200:	learn: 0.5704325	test: 0.5851712	best: 0.5851712 (200)	total: 3.07s	remaining: 11.5s
400:	learn: 0.5379063	test: 0.5596452	best: 0.5596452 (400)	total: 6.47s	remaining: 8.94s
600:	learn: 0.52279

In [13]:
# get cv score and final predictions
score = np.mean(scores)
final_preds = np.mean(np.array(preds), axis=0)

In [14]:
# save validation predictions
final_preds_val = np.mean(np.array(preds_val), axis=0)
val_path = './training_files/valid_preds/'
os.makedirs(val_path, exist_ok=True)
pd.Series(final_preds_val).to_csv(val_path + f'{cfg.modelname}.csv')

In [15]:
# save predictions
sub = pd.read_csv('../data/raw/sample_submission.csv', index_col=0)
sub[TARGET] = final_preds
sub.head()

out_path = '../submissions/'
os.makedirs(out_path, exist_ok=True)
sub.to_csv(out_path + f'{cfg.modelname}.csv')